<a href="https://colab.research.google.com/github/szperan21/BasicPythonMiniProject/blob/main/NumpyMatplotlibMiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import sys
import datetime
import math
from mpi4py import MPI

ModuleNotFoundError: No module named 'mpi4py'

In [ ]:
def inside_circle(total_count):
    x = np.random.uniform(size=total_count)
    y = np.random.uniform(size=total_count)
    radii = np.sqrt(x*x + y*y)
    count = len(radii[np.where(radii<=1.0)])
    return count


In [ ]:
comm = MPI.COMM_WORLD
cpus = comm.Get_size()
rank = comm.Get_rank()

for loopControl in range(2,10):
  n_samples = 10**loopControl

  if rank == 0:
    start_time = datetime.datetime.now()
    partitions = [ int(n_samples / cpus) ] * cpus
    counts = [ int(0) ] * cpus
  else:
    partitions = None
    counts = None

  partition_item = comm.scatter(partitions, root=0)
  count_item = inside_circle(partition_item)
  counts = comm.gather(count_item, root=0)

  if rank == 0:
    my_pi = 4.0 * counts / n_samples
    size_of_float = np.dtype(np.float64).itemsize
    memory_required = 3 * n_samples * size_of_float / (1024**3)
    end_time = datetime.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    relative_error = abs(math.pi-my_pi)/math.pi
    print("N:", n_samples, " Pi:", my_pi, " Memory:", memory_required, "GiB", " Time:", elapsed_time, "s", "Relative Error:", relative_error)


N: 100  Pi: 3.28  Memory: 2.2351741790771484e-06 GiB  Time: 0.000191 s Relative Error: 0.04405642668283338
N: 1000  Pi: 3.084  Memory: 2.2351741790771484e-05 GiB  Time: 0.00053 s Relative Error: 0.018332311009189507
N: 10000  Pi: 3.1528  Memory: 0.00022351741790771484 GiB  Time: 0.00077 s Relative Error: 0.003567409160255283
N: 100000  Pi: 3.14136  Memory: 0.0022351741790771484 GiB  Time: 0.010877 s Relative Error: 7.405593768724863e-05
N: 1000000  Pi: 3.140732  Memory: 0.022351741790771484 GiB  Time: 0.049548 s Relative Error: 0.00027395454621076306
N: 10000000  Pi: 3.141362  Memory: 0.22351741790771484 GiB  Time: 0.431393 s Relative Error: 7.341931791493343e-05
N: 100000000  Pi: 3.14162028  Memory: 2.2351741790771484 GiB  Time: 5.213201 s Relative Error: 8.793759488668921e-06
